In [1]:
import pandas as pd
from funcs import *
import warnings
warnings.filterwarnings("ignore")

### 候选变量合并

In [2]:
#季度特征
feature_base =pd.read_csv('./data/features_card.csv',index_col=0)
feature_trade = pd.read_csv('./new_out/3m_feature_trade.csv',index_col=0)
feature_loan = pd.read_csv('./new_out/3m_loan_all.csv')
tmp = feature_loan[['客户','起算日期','应收单号','收款状态','zq_day','yq_day','季度逾期占比', '季度提前还款占比', '季度平均逾期天数']]
feature_all = feature_trade.merge(tmp,left_index=True,right_on='应收单号',how='left')
feature_all =feature_base.merge(feature_all,left_index=True,right_on='客户',how='right')
feature_all.shape

(1430, 51)

In [3]:
#所有历史特征
loan_all =pd.read_csv('./data/loan_all.csv')
trade_all =pd.read_csv('./data/zq_feature_trade.csv',index_col=0)
tmp =trade_all.merge(loan_all[['4天以内占比','5-10天占比','11天以上占比','逾期占比','提前还款占比','平均逾期天数','应收单号']],
                     left_index=True,right_on='应收单号',how='left')
tmp.columns = pd.Series(tmp.columns).apply(lambda x:'历史_'+x if x!='应收单号' else x)
feature_all =feature_all.merge(tmp,on='应收单号',how='left')
feature_all =feature_all[feature_all['收款状态']=='已收款']
feature_all.shape

(1391, 84)

In [4]:
#交叉特征
feature_all['订单时间间隔均值变化'] = feature_all['订单时间间隔均值（天）'] /feature_all['历史_订单时间间隔均值（天）']
feature_all['交易额均值变化'] = feature_all['交易额均值'] /feature_all['历史_交易额均值']
feature_all['交易稳定性变化'] = feature_all['交易稳定性']/feature_all['历史_交易稳定性']
feature_all['账期订单交易量均值变化'] = feature_all['账期订单交易量均值'] /feature_all['历史_账期订单交易量均值']
feature_all['账期订单交易额均值变化'] = feature_all['账期订单交易额均值'] /feature_all['历史_账期订单交易额均值']
feature_all['逾期占比变化'] = feature_all['季度逾期占比'] /feature_all['历史_逾期占比']
feature_all['提前还款占比变化'] = feature_all['季度提前还款占比'] /feature_all['历史_提前还款占比']
feature_all['平均逾期天数变化'] = feature_all['季度平均逾期天数'] /feature_all['历史_平均逾期天数']
feature_all.shape

(1391, 92)

In [6]:
#排序,打标签
tmp_col = ['客户','收款状态','yq_day','起算日期']
feature_all =feature_all.sort_values(by='起算日期')
zq_3m =label_zq(feature_all,tmp_col)
zq_3m.to_csv('./out1/samples_all_88.csv')
zq_3m.shape,zq_3m.columns

((1391, 88),
 Index([u'注册地址', u'注册资本变更', u'法人变更', u'分支机构', u'商标专利', u'场地归属', u'有无贷款',
        u'企业征信', u'涉诉信息', u'下游客户情况', u'注册时间', u'实收资本', u'员工人数', u'厂房面积',
        u'资产负债率', u'流动比率', u'净资产', u'货币资金', u'存货周转天数', u'应收账款周转天数', u'主营业务利润率',
        u'净资产增长率', u'销售增长率', u'当地经济环境', u'撮合单数', u'订单总数', u'订单时间间隔均值（天）',
        u'订单时间间隔方差（天）', u'交易总量', u'交易量最大值', u'交易量最小值', u'交易量方差', u'交易量均值',
        u'交易额总量', u'交易额最大值', u'交易额最小值', u'交易额方差', u'交易额均值', u'交易稳定性', u'账期订单占比',
        u'账期订单交易量均值', u'账期订单交易额均值', u'zq_day', u'季度逾期占比', u'季度提前还款占比',
        u'季度平均逾期天数', u'历史_订单总数', u'历史_合作时间（月）', u'历史_最近交易（X月之前）',
        u'历史_最近连续交易月数', u'历史_采购断档总月数', u'历史_订单时间间隔均值（天）', u'历史_订单时间间隔方差（天）',
        u'历史_交易总量', u'历史_交易量最大值', u'历史_交易量最小值', u'历史_交易量方差', u'历史_交易量均值',
        u'历史_交易额总量', u'历史_交易额最大值', u'历史_交易额最小值', u'历史_交易额方差', u'历史_交易额均值',
        u'历史_交易稳定性', u'历史_供应商数量', u'历史_采购产品种类数量', u'历史_账期订单占比', u'历史_账期订单交易量均值',
        u'历史_账期订单交易额均值', u'历史_账期合作时间（月）', u'历史_最近一笔账期订单（X月之前）',
        u'历史_采购频率（订单总数/

In [7]:
#变量分析
feature_sts = stastic(zq_3m)
feature_sts

,count,mean,std,min,25%,50%,75%,max,null_ratio,ivs,pearson_label
注册地址,1331.0,5.762585e-01,4.943362e-01,0.000000,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.043134,0.091191,0.100858
注册资本变更,1331.0,1.382419e-01,3.452835e-01,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.043134,0.012971,0.039383
法人变更,1331.0,4.658152e-02,2.108200e-01,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.043134,0.001012,-0.000621
分支机构,1331.0,9.090909e-02,2.875878e-01,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.043134,0.001803,0.009891
商标专利,1331.0,6.198347e-01,4.856097e-01,0.000000,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.043134,0.092123,0.100191
场地归属,1331.0,3.471074e-01,4.762292e-01,0.000000,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.043134,0.118460,0.121791
有无贷款,1244.0,7.974277e-01,4.020780e-01,0.000000,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.105679,0.039329,0.066343
企业征信,992.0,1.508065e+00,8.172802e-01,1.000000,1.000000e+00,1.000000e+00,2.000000e+00,4.000000e+00,0.286844,0.236759,0.112246
涉诉信息,1331.0,3.688956e-01,6.682219e-01,0.000000,0.000000e+00,0.000000e+00,1.000000e+00,4.000000e+00,0.043134,0.068035,0.057467
下游客户情况,1202.0,2.572379e+00,9.988349e-01,1.000000,2.000000e+00,2.000000e+00,4.000000e+00,4.000000e+00,0.135873,0.323262,-0.146150


### 提取重要特征

In [9]:
#woe编码,归一化，缺失值填充
def pre_trate(sub_cols,df):
    df_sub = df[sub_cols].copy()
    #woe 编码
    woe_col = [col for col in sub_cols if df[col].unique().size<10]
    rate_col = [x for x in sub_cols if df[x].max()>10]    
    for col in woe_col:
        if col !='label':            
            woe_df = calc_woe(df_sub[[col,'label']],col)
            woe_dic = woe_to_dict(woe_df)
            df_sub[col] = df_sub[col].map(woe_dic)
    #归一化    
    for col in rate_col:
        df_sub[col] =normalize(df_sub[col])   
        
    #缺失值
    for col in sub_cols:
        df_sub[col] =df_sub[col].fillna(df_sub[col].mean())
    return df_sub

In [10]:
cols_filter =['实收资本','注册地址','场地归属','注册时间','下游客户情况','涉诉信息','有无贷款','企业征信','净资产','流动比率',
              '资产负债率','货币资金','主营业务利润率','应收账款周转天数','存货周转天数','季度逾期占比','季度平均逾期天数',
              '历史_逾期占比','历史_提前还款占比','季度提前还款占比','历史_平均逾期天数','平均逾期天数变化','逾期占比变化',
              '订单总数','交易额总量','历史_交易稳定性','交易量最小值','交易总量','交易额最小值','交易额方差','交易量方差',
              '提前还款占比变化','历史_账期订单交易量均值','历史_交易额均值','历史_账期订单交易额均值','交易额最大值','交易额均值',
              '交易稳定性','历史_最近连续交易月数','历史_账期合作时间（月）','label']
df = pre_trate(cols_filter,zq_3m)
df.to_csv('./out1/samples_41_fillna_mean_time.csv')

In [11]:
df_sub = zq_3m[cols_filter]
df_sub.to_csv('./out1/samples_41_for_card_init.csv')

In [13]:
#季度订单所有变量
cols_filter = ['注册地址','注册资本变更','法人变更','分支机构','商标专利','场地归属','有无贷款','企业征信','涉诉信息','下游客户情况',
               '注册时间','实收资本','员工人数','厂房面积','资产负债率','流动比率','净资产','货币资金','存货周转天数',
               '应收账款周转天数','主营业务利润率','净资产增长率','销售增长率','当地经济环境','撮合单数','订单总数',
               '订单时间间隔均值（天）','订单时间间隔方差（天）','交易总量','交易量最大值','交易量最小值','交易量方差',
               '交易量均值','交易额总量','交易额最大值','交易额最小值','交易额方差','交易额均值','交易稳定性','账期订单占比',
               '账期订单交易量均值','账期订单交易额均值','季度逾期占比','季度提前还款占比','季度平均逾期天数',
               '历史_最近连续交易月数','历史_订单时间间隔均值（天）','历史_交易额均值','历史_交易稳定性','历史_账期订单交易量均值',
               '历史_账期订单交易额均值','历史_账期合作时间（月）','历史_逾期占比','历史_提前还款占比','历史_平均逾期天数',
               '订单时间间隔均值变化','交易额均值变化','交易稳定性变化','账期订单交易量均值变化','账期订单交易额均值变化','逾期占比变化',
               '提前还款占比变化','平均逾期天数变化','zq_day','label']

In [14]:
#spss离散之后woe编码
def trans_int(x):
    try:
        return int(x)
    except:
        return np.nan
samples_spss =pd.read_csv('./out1/samples_spss_bin_sort_time.csv',index_col=0)
for col in samples_spss.columns:
    samples_spss[col] =samples_spss[col].apply(trans_int)
#add 0914
tmp = ['历史_交易总量','历史_交易额总量','订单时间间隔均值变化','账期订单交易量均值变化','账期订单交易额均值变化']
samples_spss[tmp]=zq_3m[tmp]    
for col in ['历史_账期合作时间（月）','历史_最近连续交易月数']+tmp:
    tmp,sf = bins_num(samples_spss[[col,'label']],col,3)
    samples_spss[col] =sf
for col in samples_spss.columns:
    if col!='label':
        woe_df = calc_woe(samples_spss[[col,'label']],col)
        print col
        print woe_df
        woe_dic = woe_to_dict(woe_df)
        samples_spss[col] =samples_spss[col].map(woe_dic)
samples_spss.head()

实收资本
   name  good_num  bad_num  good_ratio        iv       woe  bad_ratio
0   1.0       430       48    0.356846  0.032015 -0.324096   0.258065
1   2.0       403       59    0.334440  0.000912 -0.052911   0.317204
2   3.0       325       66    0.269710  0.023352  0.274318   0.354839
注册地址
   name  good_num  bad_num  good_ratio        iv       woe  bad_ratio
0   0.0       513       51    0.425726  0.066669 -0.439962   0.274194
1   1.0       645      122    0.535270  0.024522  0.203259   0.655914
场地归属
   name  good_num  bad_num  good_ratio        iv       woe  bad_ratio
0   0.0       782       87    0.648963  0.059342 -0.327458   0.467742
1   1.0       376       86    0.312033  0.059118  0.393246   0.462366
注册时间
   name  good_num  bad_num  good_ratio        iv       woe  bad_ratio
0   1.0       420       55    0.348548  0.008690 -0.164433   0.295699
1   2.0       399       48    0.331120  0.018211 -0.249272   0.258065
2   3.0       339       70    0.281328  0.027648  0.290983   0.376344


,实收资本,注册地址,场地归属,注册时间,下游客户情况,涉诉信息,有无贷款,企业征信,净资产,流动比率,...,交易额均值,交易稳定性,历史_最近连续交易月数,历史_账期合作时间（月）,label,历史_交易总量,历史_交易额总量,订单时间间隔均值变化,账期订单交易量均值变化,账期订单交易额均值变化
应收单号,,,,,,,,,,,,,,,,,,,,,
YS2016030800082,0.274318,-0.439962,0.393246,0.290983,-0.484329,0.070214,0.079079,-1.742430,0.139249,-0.501472,...,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
YS2016030400009,-0.324096,-0.439962,0.393246,-0.164433,-0.086596,-0.093581,-0.478549,NaN,-0.546015,-0.501472,...,-0.393759,-0.419708,-0.320472,-0.201534,0,-0.132992,-0.286177,NaN,NaN,NaN
YS2016030400024,0.274318,0.203259,0.393246,0.290983,1.757263,1.329492,0.079079,-0.226458,0.139249,0.632134,...,-0.077422,-0.419708,-0.320472,-0.201534,0,-0.132992,-0.286177,0.094295,-0.123942,0.162763
YS2016030700026,-0.324096,0.203259,-0.327458,-0.249272,-0.086596,-0.093581,-0.478549,NaN,-0.546015,-0.798740,...,-0.393759,0.341359,0.270350,-0.201534,0,0.350617,-0.033619,0.094295,-0.123942,0.162763
YS2016030700041,0.274318,0.203259,0.393246,0.290983,1.757263,1.329492,0.079079,-0.226458,0.139249,0.632134,...,-0.077422,-0.419708,-0.320472,-0.201534,0,-0.132992,-0.286177,0.094295,-0.123942,0.162763


In [18]:
#保存woe
df_mu =pd.DataFrame()
def trans_int(x):
    try:
        return int(x)
    except:
        return np.nan
samples_spss =pd.read_csv('./out1/samples_spss_bin_sort_time.csv',index_col=0)
for col in samples_spss.columns:
    samples_spss[col] =samples_spss[col].apply(trans_int)
for col in ['历史_账期合作时间（月）','历史_最近连续交易月数']:
    tmp,sf = bins_num(samples_spss[[col,'label']],col,3)
    samples_spss[col] =sf
for col in samples_spss.columns:
    if col!='label':
        woe_df = calc_woe(samples_spss[[col,'label']],col)
        woe_df['col'] = [col]*woe_df.shape[0]
        df_mu=pd.concat([df_mu,woe_df[['name','col','woe']]])
df_mu.to_csv('./out1/woe_cols.csv',index=None)        
df_mu

,name,col,woe
0,1.0,实收资本,-0.324096
1,2.0,实收资本,-0.052911
2,3.0,实收资本,0.274318
0,0.0,注册地址,-0.439962
1,1.0,注册地址,0.203259
0,0.0,场地归属,-0.327458
1,1.0,场地归属,0.393246
0,1.0,注册时间,-0.164433
1,2.0,注册时间,-0.249272
2,3.0,注册时间,0.290983


In [9]:
#0填充
samples_spss =samples_spss.fillna(0)
samples_spss.to_csv('./out1/samples_for_card_fillna0.csv')
samples_spss

,实收资本,注册地址,场地归属,注册时间,下游客户情况,涉诉信息,有无贷款,企业征信,净资产,流动比率,...,提前还款占比变化,历史_账期订单交易量均值,历史_交易额均值,历史_账期订单交易额均值,交易额最大值,交易额均值,交易稳定性,历史_最近连续交易月数,历史_账期合作时间（月）,label
0,-0.324096,-0.439962,-0.327458,0.290983,-0.086596,-0.093581,0.079079,1.223350,0.213717,-0.798740,...,0.369253,-0.028632,-0.446218,-0.422121,-0.458194,-0.393759,-0.419708,0.270350,0.251760,0
1,-0.324096,-0.439962,-0.327458,0.290983,-0.086596,-0.093581,0.079079,1.223350,0.213717,-0.798740,...,0.369253,-0.438963,-0.446218,-0.422121,-0.458194,-0.393759,-0.419708,0.270350,0.251760,0
2,-0.324096,-0.439962,-0.327458,0.290983,-0.086596,-0.093581,0.079079,1.223350,0.213717,-0.798740,...,0.369253,-0.438963,-0.446218,-0.422121,-0.458194,-0.393759,-0.419708,0.270350,0.251760,0
3,-0.324096,-0.439962,-0.327458,0.290983,-0.086596,-0.093581,0.079079,1.223350,0.213717,-0.798740,...,-0.182491,-0.438963,-0.446218,-0.422121,-0.458194,-0.393759,-0.419708,-0.130604,-0.201534,0
4,-0.052911,-0.439962,0.393246,0.290983,-0.086596,0.070214,0.079079,1.223350,0.139249,0.632134,...,-0.693111,0.371127,-0.077422,-0.095122,0.058664,-0.077422,-0.002052,-0.130604,0.251760,1
5,-0.052911,-0.439962,0.393246,0.290983,-0.086596,0.070214,0.079079,1.223350,0.139249,0.632134,...,-0.693111,0.371127,-0.077422,-0.095122,0.058664,-0.077422,-0.002052,-0.130604,0.251760,1
6,-0.052911,-0.439962,0.393246,0.290983,-0.086596,0.070214,0.079079,1.223350,0.139249,0.632134,...,-0.182491,-0.438963,-0.077422,-0.095122,0.058664,-0.077422,-0.419708,-0.130604,0.251760,0
7,-0.052911,-0.439962,0.393246,0.290983,-0.086596,0.070214,0.079079,1.223350,0.139249,0.632134,...,-0.182491,-0.438963,-0.077422,-0.095122,0.058664,-0.077422,-0.419708,-0.130604,0.251760,0
8,-0.052911,-0.439962,0.393246,0.290983,-0.086596,0.070214,0.079079,1.223350,0.139249,0.632134,...,-0.182491,0.371127,-0.077422,-0.095122,0.058664,-0.077422,-0.419708,-0.130604,0.251760,0
9,-0.052911,-0.439962,0.393246,0.290983,-0.086596,0.070214,0.079079,1.223350,0.139249,0.632134,...,-0.182491,0.371127,-0.077422,-0.095122,0.058664,-0.077422,-0.002052,-0.130604,0.251760,0


In [15]:
#均值填充
for col in samples_spss.columns:
    samples_spss[col]=samples_spss[col].fillna(samples_spss[col].mean())
tmp=['实收资本', '注册地址', '场地归属', '注册时间', '下游客户情况', '涉诉信息', '有无贷款','企业征信', '净资产', '流动比率', '资产负债率', '货币资金', '主营业务利润率', '应收账款周转天数','存货周转天数', '季度逾期占比', '季度平均逾期天数', '历史_逾期占比', '历史_提前还款占比','季度提前还款占比', '历史_平均逾期天数', '平均逾期天数变化', '逾期占比变化', '订单总数', '交易额总量','历史_交易稳定性', '交易量最小值', '交易总量', '交易额最小值', '交易额方差', '交易量方差','提前还款占比变化', '历史_账期订单交易量均值', '历史_交易额均值', '历史_账期订单交易额均值', '交易额最大值','交易额均值', '交易稳定性', '历史_最近连续交易月数', '历史_账期合作时间（月）', '历史_交易总量', '历史_交易额总量', '订单时间间隔均值变化', '账期订单交易量均值变化', '账期订单交易额均值变化','label']
samples_spss =samples_spss[tmp]
samples_spss.to_csv('./out1/samples_for_card_fillna_mean.csv')
samples_spss

,实收资本,注册地址,场地归属,注册时间,下游客户情况,涉诉信息,有无贷款,企业征信,净资产,流动比率,...,交易额均值,交易稳定性,历史_最近连续交易月数,历史_账期合作时间（月）,历史_交易总量,历史_交易额总量,订单时间间隔均值变化,账期订单交易量均值变化,账期订单交易额均值变化,label
应收单号,,,,,,,,,,,,,,,,,,,,,
YS2016030800082,0.274318,-0.439962,0.393246,0.290983,-0.484329,0.070214,0.079079,-1.742430,0.139249,-0.501472,...,-0.029661,-0.027321,-0.012667,-0.006778,-0.017724,-0.012850,0.032428,0.003712,-0.015402,1
YS2016030400009,-0.324096,-0.439962,0.393246,-0.164433,-0.086596,-0.093581,-0.478549,-0.027717,-0.546015,-0.501472,...,-0.393759,-0.419708,-0.320472,-0.201534,-0.132992,-0.286177,0.032428,0.003712,-0.015402,0
YS2016030400024,0.274318,0.203259,0.393246,0.290983,1.757263,1.329492,0.079079,-0.226458,0.139249,0.632134,...,-0.077422,-0.419708,-0.320472,-0.201534,-0.132992,-0.286177,0.094295,-0.123942,0.162763,0
YS2016030700026,-0.324096,0.203259,-0.327458,-0.249272,-0.086596,-0.093581,-0.478549,-0.027717,-0.546015,-0.798740,...,-0.393759,0.341359,0.270350,-0.201534,0.350617,-0.033619,0.094295,-0.123942,0.162763,0
YS2016030700041,0.274318,0.203259,0.393246,0.290983,1.757263,1.329492,0.079079,-0.226458,0.139249,0.632134,...,-0.077422,-0.419708,-0.320472,-0.201534,-0.132992,-0.286177,0.094295,-0.123942,0.162763,0
YS2016030800064,-0.052911,-0.439962,0.393246,0.290983,-0.101888,-0.093581,-0.033881,-0.027717,0.139249,-0.798740,...,-0.077422,-0.419708,-0.320472,-0.201534,-0.132992,-0.286177,0.032428,0.003712,-0.015402,0
YS2016030800101,-0.324096,0.203259,0.393246,-0.164433,-0.086596,-0.093581,0.079079,-0.226458,-0.546015,-0.501472,...,-0.077422,-0.419708,-0.130604,-0.201534,-0.132992,-0.286177,0.094295,0.003712,-0.015402,0
YS2016030700034,-0.324096,-0.439962,0.393246,-0.249272,-0.484329,-0.093581,-0.478549,-0.027717,-0.546015,0.632134,...,-0.393759,-0.419708,-0.130604,-0.201534,-0.132992,-0.286177,0.094295,0.003712,-0.015402,0
YS2016031100279,0.274318,0.203259,0.393246,-0.164433,-0.484329,-0.093581,0.079079,-0.226458,0.139249,0.632134,...,0.383103,0.341359,-0.320472,-0.201534,-0.258285,-0.033619,0.094295,-0.123942,0.162763,1


In [13]:
samples_spss.columns

Index([u'实收资本', u'注册地址', u'场地归属', u'注册时间', u'下游客户情况', u'涉诉信息', u'有无贷款',
       u'企业征信', u'净资产', u'流动比率', u'资产负债率', u'货币资金', u'主营业务利润率', u'应收账款周转天数',
       u'存货周转天数', u'季度逾期占比', u'季度平均逾期天数', u'历史_逾期占比', u'历史_提前还款占比',
       u'季度提前还款占比', u'历史_平均逾期天数', u'平均逾期天数变化', u'逾期占比变化', u'订单总数', u'交易额总量',
       u'历史_交易稳定性', u'交易量最小值', u'交易总量', u'交易额最小值', u'交易额方差', u'交易量方差',
       u'提前还款占比变化', u'历史_账期订单交易量均值', u'历史_交易额均值', u'历史_账期订单交易额均值', u'交易额最大值',
       u'交易额均值', u'交易稳定性', u'历史_最近连续交易月数', u'历史_账期合作时间（月）', u'label',
       u'历史_交易总量', u'历史_交易额总量', u'订单时间间隔均值变化', u'账期订单交易量均值变化', u'账期订单交易额均值变化'],
      dtype='object')

### 老客户样本

In [23]:
indexs = ['订单总数','订单时间间隔均值（天）','订单时间间隔方差（天）','交易总量','交易量最大值','交易量最小值','交易量方差',
          '交易量均值','交易额总量','交易额最大值','交易额最小值','交易额方差','交易额均值','交易稳定性',
          '账期订单占比','账期订单交易量均值','账期订单交易额均值']
old_cos = zq_3m.dropna(subset=indexs)
old_cos.to_csv('./new_out/old_custom_samples_3m.csv')
old_cos.shape

(1288, 65)

In [24]:
#季度-老客户
old_cos_3m = fillna_nor(old_cos)
old_cos_3m.to_csv('./new_out/old_custom_samples_3m_fillna_nor.csv',index=None)
old_cos_3m.head()

,注册地址,注册资本变更,法人变更,分支机构,商标专利,场地归属,有无贷款,企业征信,涉诉信息,下游客户情况,...,历史_平均逾期天数,订单时间间隔均值变化,交易额均值变化,交易稳定性变化,账期订单交易量均值变化,账期订单交易额均值变化,逾期占比变化,提前还款占比变化,平均逾期天数变化,label
10,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,2.0,...,0.046875,0.0,0.371899,0.427693,0.359601,0.282888,0.027778,-0.222222,0.368852,0
11,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,2.0,...,0.046875,0.0,0.380545,0.427962,0.371286,0.294310,0.000000,-0.250000,0.368852,0
12,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,2.0,...,0.046875,0.0,0.379655,0.427875,0.371286,0.294310,0.047619,-0.285714,0.368852,0
14,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,2.0,...,0.062500,0.0,0.388602,0.428424,0.371286,0.294310,0.000000,0.000000,0.393443,0
27,0.0,0.0,0.0,1.0,1.0,1.0,1.0,3.0,1.0,2.0,...,0.171875,0.0,0.370446,0.428874,0.345199,0.272994,-0.125000,0.166667,0.467213,1
28,0.0,0.0,0.0,1.0,1.0,1.0,1.0,3.0,1.0,2.0,...,0.156250,0.0,0.370446,0.428874,0.345199,0.272994,-0.168831,0.207792,0.467213,1
29,0.0,0.0,0.0,1.0,1.0,1.0,1.0,3.0,1.0,2.0,...,0.156250,0.0,0.381625,0.427586,0.371286,0.294310,0.000000,0.000000,0.393443,0
30,0.0,0.0,0.0,1.0,1.0,1.0,1.0,3.0,1.0,2.0,...,0.156250,0.0,0.381625,0.427586,0.371286,0.294310,0.000000,0.000000,0.393443,0
31,0.0,0.0,0.0,1.0,1.0,1.0,1.0,3.0,1.0,2.0,...,0.156250,0.0,0.387302,0.427460,0.371286,0.294310,0.000000,0.000000,0.393443,0
32,0.0,0.0,0.0,1.0,1.0,1.0,1.0,3.0,1.0,2.0,...,0.156250,0.0,0.399162,0.428234,0.371286,0.294310,0.000000,0.000000,0.393443,0


### 订单样本

In [9]:
#账期订单特征
feature_trade = pd.read_csv('./data/zq_feature_trade.csv',index_col=0)
feature_loan = pd.read_csv('./data/loan_all.csv')
feature_base = pd.read_csv('./data/features_card.csv',index_col=0)
tmp = feature_loan[['应收单号','客户','收款状态','4天以内占比', '5-10天占比', '11天以上占比','逾期占比','提前还款占比','yq_day',
                    'zq_day','平均逾期天数']]
feature_all = feature_trade.merge(tmp,left_index=True,right_on='应收单号',how='left')
feature_all =feature_base.merge(feature_all,left_index=True,right_on='客户',how='right')
feature_all.shape

(1430, 63)

In [10]:
#label_分账期
feature_all =feature_all[feature_all['收款状态']=='已收款']
tmp_col = ['客户','应收单号','收款状态','yq_day']
zq_all =label_zq(feature_all,tmp_col)
zq_all.to_csv('./new_out/samples_all.csv')
zq_all.shape

/home/sf/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/sf/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/sf/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.

(1391, 60)

In [23]:
#订单样本
zq_all_nor = fillna_nor(zq_all)
zq_all_nor.to_csv('./new_out/samples_all_fillna_nor.csv',index=None)

In [11]:
#老客户样本
indexs = ['订单总数','合作时间（月）','最近交易（X月之前）','最近连续交易月数','采购断档总月数','订单时间间隔均值（天）','订单时间间隔方差（天）',
          '交易总量','交易量最大值','交易量最小值','交易量方差','交易量均值','交易额总量','交易额最大值','交易额最小值','交易额方差','交易额均值',
          '交易稳定性','供应商数量','采购产品种类数量',
          '账期订单占比','账期订单交易量均值','账期订单交易额均值','账期合作时间（月）','最近一笔账期订单（X月之前）']
old_cos_all = zq_all.dropna(subset=indexs)
old_cos_all.to_csv('./new_out/old_custom_samples_all.csv')
old_cos_all.shape

(1226, 60)

In [24]:
#老客户
old_cos_nor = fillna_nor(old_cos_all)
old_cos_nor.to_csv('./new_out/old_custom_samples_all_fillna_nor.csv',index=None)
old_cos_nor.shape

(1226, 60)